In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/DataSpark/fact_sales_merge_1.csv')

df.head()

,OrderNo,Line Item,Odate,DDate,CustKey,StoreKey,ProdKey,Quant,CurCode,Gender,...,Color,UnitCostUSD,UnitPriceUSD,SubCatKey,Subcategory,CatKey,Category,SqMeters,OpenDate,ExchangeRate
0,366000,1,01-01-2016,NaN,265598,10,1304,1,CAD,Male,...,White,31.27,68.00,406,Cameras & Camcorders Accessories,4,Cameras and camcorders,NaN,NaN,1.3884
1,366001,1,01-01-2016,13-01-2016,1269051,0,1048,2,USD,Male,...,Silver,141.47,427.00,402,Digital SLR Cameras,4,Cameras and camcorders,NaN,NaN,1.0000
2,366001,2,01-01-2016,13-01-2016,1269051,0,2007,1,USD,Male,...,Black,220.64,665.94,803,Microwaves,8,Home Appliances,NaN,NaN,1.0000
3,366002,1,01-01-2016,12-01-2016,266019,0,1106,7,CAD,Female,...,Orange,148.08,322.00,402,Digital SLR Cameras,4,Cameras and camcorders,NaN,NaN,1.3884
4,366002,2,01-01-2016,12-01-2016,266019,0,373,1,CAD,Female,...,White,166.20,326.00,301,Laptops,3,Computers,NaN,NaN,1.3884


In [ ]:
#check for missing values
df.isnull().sum()

,0
OrderNo,0
Line Item,0
Odate,0
DDate,49719
CustKey,0
StoreKey,0
ProdKey,0
Quant,0
CurCode,0
Gender,0


In [ ]:
#filling the missing values in SqMeters and OpenDate Column by referring to store key, state values
df['SqMeters'].fillna(df.groupby('StoreKey')['SqMeters'].transform('mean'), inplace=True)
df['OpenDate'].fillna(df.groupby('StoreKey')['OpenDate'].transform('mean'), inplace=True)

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
#check the datatypes of all columns
df.dtypes

,0
OrderNo,int64
Line Item,int64
Odate,object
DDate,object
CustKey,int64
StoreKey,int64
ProdKey,int64
Quant,int64
CurCode,object
Gender,object


In [ ]:
# Fill missing 'SqMeters' with the mean of each group ('StoreKey', 'State')
df['SqMeters'] = df.groupby(['StoreKey', 'State'])['SqMeters'].transform(lambda x: x.fillna(x.mean()))

# For 'OpenDate', fill with the most frequent date within the group
df['OpenDate'] = df.groupby(['StoreKey', 'State'])['OpenDate'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else pd.NaT))

# If there are still missing values, apply global imputation
df['SqMeters'].fillna(df['SqMeters'].mean(), inplace=True)
df['OpenDate'].fillna(df['OpenDate'].mode()[0], inplace=True)

# Verify if the missing values are resolved
df[['SqMeters', 'OpenDate']].isnull().sum()


,0
SqMeters,0
OpenDate,0


In [ ]:
df.isnull().sum()

,0
OrderNo,0
Line Item,0
Odate,0
DDate,49719
CustKey,0
StoreKey,0
ProdKey,0
Quant,0
CurCode,0
Gender,0


In [ ]:
import pandas as pd

# Convert 'Odate' and 'DDate' to datetime format if they aren't already
df['Odate'] = pd.to_datetime(df['Odate'], dayfirst=True) # Added dayfirst=True to handle DD-MM-YYYY format
df['DDate'] = pd.to_datetime(df['DDate'], dayfirst=True) # Added dayfirst=True for consistency

# Calculate the difference between 'DDate' and 'Odate' where 'DDate' is available
df['Delivery_Days'] = (df['DDate'] - df['Odate']).dt.days

# Calculate the median delivery time
median_delivery_days = df['Delivery_Days'].median()

# Fill missing 'DDate' values by adding the median delivery time to 'Odate'
df['DDate'].fillna(df['Odate'] + pd.to_timedelta(median_delivery_days, unit='D'), inplace=True)

# Drop the temporary 'Delivery_Days' column
df.drop(columns=['Delivery_Days'], inplace=True)

# Verify the missing values in 'DDate' are filled
df['DDate'].isnull().sum()

0

In [ ]:
df.isnull().sum()

,0
OrderNo,0
Line Item,0
Odate,0
DDate,0
CustKey,0
StoreKey,0
ProdKey,0
Quant,0
CurCode,0
Gender,0


In [ ]:
#save to csv
df.to_csv('DM_fact_sales_merged_data.csv', index=False)

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/DataSpark/DM_fact_sales_merged_data.csv')
df.head()

,OrderNo,Line Item,Odate,DDate,CustKey,StoreKey,ProdKey,Quant,CurCode,Gender,...,Color,UnitCostUSD,UnitPriceUSD,SubCatKey,Subcategory,CatKey,Category,SqMeters,OpenDate,ExchangeRate
0,366000,1,2016-01-01,2016-01-05,265598,10,1304,1,CAD,Male,...,White,31.27,68.00,406,Cameras & Camcorders Accessories,4,Cameras and camcorders,1210.000000,01-01-2010,1.3884
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,Male,...,Silver,141.47,427.00,402,Digital SLR Cameras,4,Cameras and camcorders,1582.342707,01-01-2010,1.0000
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,Male,...,Black,220.64,665.94,803,Microwaves,8,Home Appliances,1582.342707,01-01-2010,1.0000
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,Female,...,Orange,148.08,322.00,402,Digital SLR Cameras,4,Cameras and camcorders,1582.342707,01-01-2010,1.3884
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,Female,...,White,166.20,326.00,301,Laptops,3,Computers,1582.342707,01-01-2010,1.3884


In [ ]:
#check datatype of all coulmns
df.dtypes

,0
OrderNo,int64
Line Item,int64
Odate,object
DDate,object
CustKey,int64
StoreKey,int64
ProdKey,int64
Quant,int64
CurCode,object
Gender,object


In [ ]:
#change the data type of OpenDate Column to date Data Type
df['OpenDate'] = pd.to_datetime(df['OpenDate'], format='%d-%m-%Y') # Specify the correct format

In [ ]:
#change name of columns Line Item, Birth Month, Birth Year to LineItem, BirthMonth, BirthYear
df.rename(columns={'Line Item': 'LineItem'}, inplace=True)
df.rename(columns={'Birth Month' : 'BirthMonth'}, inplace=True)
df.rename(columns={'Birth Year' : 'BirthYear'}, inplace=True)

df.head()

,OrderNo,LineItem,Odate,DDate,CustKey,StoreKey,ProdKey,Quant,CurCode,Gender,...,Color,UnitCostUSD,UnitPriceUSD,SubCatKey,Subcategory,CatKey,Category,SqMeters,OpenDate,ExchangeRate
0,366000,1,2016-01-01,2016-01-05,265598,10,1304,1,CAD,Male,...,White,31.27,68.00,406,Cameras & Camcorders Accessories,4,Cameras and camcorders,1210.000000,2010-01-01,1.3884
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,Male,...,Silver,141.47,427.00,402,Digital SLR Cameras,4,Cameras and camcorders,1582.342707,2010-01-01,1.0000
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,Male,...,Black,220.64,665.94,803,Microwaves,8,Home Appliances,1582.342707,2010-01-01,1.0000
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,Female,...,Orange,148.08,322.00,402,Digital SLR Cameras,4,Cameras and camcorders,1582.342707,2010-01-01,1.3884
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,Female,...,White,166.20,326.00,301,Laptops,3,Computers,1582.342707,2010-01-01,1.3884


In [ ]:
df.dtypes

,0
OrderNo,int64
LineItem,int64
Odate,object
DDate,object
CustKey,int64
StoreKey,int64
ProdKey,int64
Quant,int64
CurCode,object
Gender,object


In [ ]:
from datetime import datetime
# Convert 'Odate' and 'DDate' to datetime format if they aren't already
df['Odate'] = pd.to_datetime(df['Odate'])
df['DDate'] = pd.to_datetime(df['DDate'])

In [ ]:
df.dtypes

,0
OrderNo,int64
LineItem,int64
Odate,datetime64[ns]
DDate,datetime64[ns]
CustKey,int64
StoreKey,int64
ProdKey,int64
Quant,int64
CurCode,object
Gender,object


In [ ]:
#convert to csv
df.to_csv('DM_fact_sales_merged_data.csv', index=False)